In [1]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});


<IPython.core.display.Javascript object>

# The Variational Quantum Factoring Algorithm 

The aim ofthe algorithim is given a number $m$ find $p$, $q$ such that $m = p \times q$. Both $p, q$ are primes. 


## Factoring as binary optimization:

- We want to set this factoring problem up as a optimization problem. So what are we to  optimize ? In the paper [variational quantum
factoring](https://arxiv.org/abs/1808.08927) the optimization is set up as the following 

The binary representations of $m, p, q$ are


$$
m=\sum_{k=0}^{n_m-1} 2^im_k\\
p=\sum_{k=0}^{n_p-1} 2^ip_k\\
q=\sum_{k=0}^{n_q-1} 2^iq_k
$$

where $m_k, q_k, p_k$ are the $k^{th}$ bits of $m,q,p$ respectively. $n_m$ is the number of bits of m and similarly for $p,q$. 


From the binary multiplication $m=p\times q$ we get the following equation [factoring as optimization](https://www.microsoft.com/en-us/research/publication/factoring-as-optimization/) ,

\begin{equation}
0 = \sum_{j=0}^{i} q_jp_{i-j} +  \sum_{j=0}^{i}z_{j,i} - m_i - \sum_{j=1}^{n_c} 2^j z_{i, i+j} \label{eq1}
\end{equation}


for all $0\le i \le n_c$. $z_{i,j}$ represents the carry  bit from position $i$ to position $j$ and $n_c = n_p + n_q -1$.

We now assign a cumber $C_i$ to the l.h.s of the above equation 

\begin{equation}
C_i = \sum_{j=0}^{i} q_jp_{i-j} +  \sum_{j=0}^{i}z_{j,i} - m_i - \sum_{j=1}^{n_c} 2^j z_{i, i+j} 
\end{equation}

and represent factoring problem as finding the set of binary variables $\{p_i\}, \{q_i\}, \{z_{i,j}\}$ which satisfy the following equation
\begin{equation}
0= \sum_{i=0}^{n_c} C_i^2 
\end{equation}

Since this is a sum of positive terms the only way the sum can be identically zero is if each term is independently zero, which gurantees each  $C_i$ is equal to zero.




## Classical preprocessing:

After this the the above equations  are simplified (i.e truncating the summation of of the final term in equation (2) above)  by applying some  classical preprocessing rules ( see [variational quantum
factoring](https://arxiv.org/abs/1808.08927))

## Constructing the Ising Hamiltonian

Suppose we have obtained $C'_i$ for each $i$ after having done the classical preprocessing of equation (2) above. The solutions of the simplified equations $C'_i=0$ then correspond to the classical minimization of the energy/cost function 

\begin{equation*}
E= \sum_{i=0}^{n_c} C_i^2 
\end{equation*}


which can be represented as a quantum factorizing Hamiltonian as
 
\begin{equation*}
H= \sum_{i=0}^{n_c} \hat{C}_i^2 
\end{equation*}


where each $\hat{C}_i$ term is obtained by the mapping  

\begin{equation*}
b_k = \frac{1}{2} (1- \sigma^z_{b,k}) 
\end{equation*}

where $b \in \{p,q,z\}$


# VQF Algorithim

- Step 1: Feed the mixing hamiltonian and the cost Hamiltonian to the QAOA circuit

- Step 2: Obtain a set of good initial parameters (both for mixing and cost hamiltonians) from output of step 1. (quantum optimization)

- Step 3: Feed the paprametrs of step 2 to the BFGS optimizer

- Step 4:  Repeat step 2 with the optimized parameters from the output of step 3. (classical optimization)

- Step 5: Repeat steps 2,3,4 until some convergence criteria is satisfied.

- step 6: Measure quantum state to obtain results

Therefore there are essentials four blocks  in the VQFT implementation.

1. Preprocessing
2. Hamiltonian creation
3. Parameter initialization (this is where quantum optimization happens )
4. Classical optimization



## Results of Implementation


Looking at the original paper [variational quantum factoring](https://arxiv.org/abs/1808.08927)) and the experiments performed by  [mstechly](https://github.com/mstechly/vqf) we find the following- 


1. As $m$ gets bigger the the number of qubits needed to factor $m = p \times q$ icreases rapidly without classical preprocessing. With classical preprocessing however the number of qubits increases much more slowly.

2. Further elaborating on the above if we don't know the length of $p, q$ the qubit scaling (with classical preprocessing)  in advance is worse that compared to the case when the  length of $p, q$ is known.

3. The closeness of the VQF results tothe exact result crucially depends of the choice of $m$. For certain numbers the VQF results are very close to the exact one, whereas for others its very bad (see FIG. 2. in [variational quantum factoring](https://arxiv.org/abs/1808.08927)).   

4. The number of BFGS function evaluations increases withthe number of circuit layers (see FIG. 3. in [variational quantum factoring](https://arxiv.org/abs/1808.08927)). (The scaling is approximately linear.) 

5. [mstechly](https://github.com/mstechly/vqf) also found that for certain numbers no matter how hard you try, you cannot find the exact solution with just one QAOA layer.

